In [1]:
import pandas as pd
import numpy as np
import env
import acquire
import warnings
warnings.filterwarnings("ignore")

# Data Acquisition & Cleaning

In [2]:
df = acquire.get_data_from_mysql()
df.head()

Type in queryselect*from customers
Name the databasetelco_churn


,customer_id,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,internet_service_type_id,online_security,...,device_protection,tech_support,streaming_tv,streaming_movies,contract_type_id,paperless_billing,payment_type_id,monthly_charges,total_charges,churn
0,0002-ORFBO,Female,0,Yes,Yes,9,Yes,No,1,No,...,No,Yes,Yes,No,2,Yes,2,65.6,593.3,No
1,0003-MKNFE,Male,0,No,No,9,Yes,Yes,1,No,...,No,No,No,Yes,1,No,2,59.9,542.4,No
2,0004-TLHLJ,Male,0,No,No,4,Yes,No,2,No,...,Yes,No,No,No,1,Yes,1,73.9,280.85,Yes
3,0011-IGKFF,Male,1,Yes,No,13,Yes,No,2,No,...,Yes,No,Yes,Yes,1,Yes,1,98.0,1237.85,Yes
4,0013-EXCHZ,Female,1,Yes,No,3,Yes,No,2,No,...,No,Yes,Yes,No,1,Yes,2,83.9,267.4,Yes


In [3]:
# tenure & monthly_charges might need to be scaled

In [3]:
df.total_charges = df.total_charges.str.strip().replace('', np.nan).astype(float)
df.drop(columns = 'customer_id', inplace = True)

In [4]:
df.fillna(np.nan, inplace=True)
df = df.dropna()

# Baseline Model

1. split train/ test
2. change churn into numerical

In [5]:
def split_my_data(data):
	from sklearn.model_selection import train_test_split
	return train_test_split(data, train_size = 0.7, random_state = 123)

In [6]:
train, test = split_my_data(df)

from sklearn.preprocessing import LabelEncoder, OneHotEncoder
# Integer Encoding
int_encoder = LabelEncoder()
train["churn"] = int_encoder.fit_transform(train["churn"])
test["churn"] = int_encoder.transform(test["churn"])

X_train = train.drop(columns = "churn")
y_train = train.churn
X_test = test.drop(columns = "churn")
y_test = test.churn

X_train = X_train.select_dtypes(int, object)
X_test = X_test.select_dtypes(int, object)

clf = DecisionTreeClassifier(criterion= 'gini',max_depth=3, random_state=123)

clf.fit(X_train, y_train)

print('Accuracy of Decision Tree classifier on training set: {:.2f}'
     .format(clf.score(X_train, y_train)))

In [7]:
def encode(train, test, col_name):
    from sklearn.preprocessing import LabelEncoder, OneHotEncoder

    encoded_values = sorted(list(train[col_name].unique()))
    
    col_name_ = col_name + '_'
    column_list_scaled = [ col_name_ + a for a in encoded_values]

    
    # Integer Encoding
    int_encoder = LabelEncoder()
    train.encoded = int_encoder.fit_transform(train[col_name])
    test.encoded = int_encoder.transform(test[col_name])
    
    # create 2D np arrays of the encoded variable (in train and test)
    train_array = np.array(train.encoded).reshape(len(train.encoded),1)
    test_array = np.array(test.encoded).reshape(len(test.encoded),1)

    # One Hot Encoding
    ohe = OneHotEncoder(sparse=False, categories='auto')
    train_ohe = ohe.fit_transform(train_array)
    test_ohe = ohe.transform(test_array)
    
    # Turn the array of new values into a data frame with columns names being the values
    # and index matching that of train/test
    # then merge the new dataframe with the existing train/test dataframe
    train_encoded = pd.DataFrame(data=train_ohe,
                            columns= column_list_scaled, index=train.index)
    train = train.join(train_encoded)
    
    test_encoded = pd.DataFrame(data=test_ohe,
                               columns=column_list_scaled, index=test.index)
    test = test.join(test_encoded)
    
    return train, test

In [8]:
for col in df.select_dtypes(object).columns:
    train, test = encode(train, test, col)

In [9]:
processed_train = train.drop(columns = train.select_dtypes(object).columns)
processed_test = test.drop(columns = test.select_dtypes(object).columns)

In [10]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

In [11]:
X_train = processed_train.drop(columns = ["churn_No","churn_Yes"])
y_train = train.churn
X_test = processed_test.drop(columns = ["churn_No","churn_Yes"])
y_test = test.churn

In [12]:
clf = DecisionTreeClassifier(criterion= 'gini',max_depth=10, random_state=123)

In [13]:
clf.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=10,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=123, splitter='best')

In [14]:
y_pred = clf.predict(X_train)

In [32]:
print('Accuracy of Decision Tree classifier on training set: {:.2f}'
     .format(clf.score(X_train, y_train)))

Accuracy of Decision Tree classifier on training set: 0.88


In [15]:
label = sorted(y_train.unique())

In [16]:
y_train.value_counts()

No     3610
Yes    1312
Name: churn, dtype: int64

In [17]:
pd.DataFrame(confusion_matrix(y_train, y_pred), index=label, columns=label)

,No,Yes
No,3345,265
Yes,336,976


In [18]:
print(classification_report(y_train, y_pred))

              precision    recall  f1-score   support

          No       0.91      0.93      0.92      3610
         Yes       0.79      0.74      0.76      1312

    accuracy                           0.88      4922
   macro avg       0.85      0.84      0.84      4922
weighted avg       0.88      0.88      0.88      4922



In [37]:
import graphviz
from graphviz import Graph
from sklearn import tree
import pydotplus
import sklearn.tree as tree
import pydotplus
import sklearn.tree as tree
from IPython.display import Image

In [38]:

# Export resulting tree to DOT source code string
dot_data = export_graphviz(clf,
                           feature_names=X_train.columns,
                           out_file=None,
                           filled=True,
                           rounded=True)

#Export to pdf
pydot_graph = pydotplus.graph_from_dot_data(dot_data)
pydot_graph.write_pdf('tree.pdf')

True

In [39]:
ds_tree = export_graphviz(clf, out_file=None) 

In [40]:
graph = graphviz.Source(ds_tree) 

In [41]:
graph.render('churn', view=True)

'churn.pdf'

In [35]:
import scipy as sp
import numpy as np

# compare the sepal_length > is continuous > t-test
sp.stats.ttest_ind(
    train[train.species == "virginica"].sepal_length,
    train[train.species == 'versicolor'].sepal_length)
# pvalue less than 0.05, reject Ho
# there is a significant difference between sepal_length of virginica and versicolor
# thus the variable sepal_length is good choice to keep as a feature to differentiate the two species

AttributeError: 'DataFrame' object has no attribute 'species'

In [ ]:
sp.stats.ttest_ind(
    train[train.species == "virginica"].sepal_width,
    train[train.species == 'versicolor'].sepal_width)

# pvalue is slightly higher than 0.05
# also much higher than sepal_length
# might be a worse option

In [ ]:
# churn rate as a whole
df["churn_rate"] = len(df[df['churn'] == 'Yes'])/ df.customer_id.nunique()

Could the sign-up month (e.g. specific promo lead to sign-up cohort) relate to customer churn? 
  Is there a cohort or cohorts who have a higher rate of churn than other cohorts? 
  (Plot the rate of churn on a line chart where x is the tenure and y is the rate of churn (customers churned/total     customers))
#form groups for tenure